In [1]:
# Импорт библиотек
import os
import time

import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWINB", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWINB", "T"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################


# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 20
Maxiterations_cotel = 4
Maxiterations_turbine = 30

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True
# Расчет для работы с теплофикацией
Teplo = int(True)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)

#####################Максимов#####################
PKM_zaryad = False
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
Enthalpies = list(map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures))


gas_streams0.loc["GTU-KU":"GPK-out", "H"] = Enthalpies

############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

# Расчет всей ПГУ вместе


def calculate_CCGT(
    Iterations_KU_TU,
    Iterations_cotel,
    Iterations_turbine,
):

    # Расчет ГТУ
    Gas_turbine_res = Gas_turbine.calc(GTU_input)

    # Запись данных об электричестве
    electric.at["GTU", "N"] = Gas_turbine_res["N"]
    electric.at["GTU", "KPD"] = Gas_turbine_res["eff"]
    electric.at["DK", "N"] = Gas_turbine_res["Ndk"]

    # Запись данных о газе на выходе из ГТУ
    gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
    gas_streams.at["GTU-KU", "T"] = Gas_turbine_res["T"]
    gas_streams.at["GTU-KU", "G"] = Gas_turbine_res["G"]
    gas_streams.at["GTU-KU", "P"] = 0.1
    gas_streams.at["GTU-KU", "H"] = gas0.p_t(
        gas_streams.at["GTU-KU", "P"], gas_streams.at["GTU-KU", "T"]
    )["h"]
    Gas_turbine_composition = pd.read_excel(
        "input.xlsx", sheet_name="Gas_composition0", index_col=0
    )
    
    #####################Максимов#####################
    from PKM import PKM_all
    PKM = PKM_all.calc(PKM_zaryad, gas_streams,syngas_streams,water_streams,water_streams0,heaters)
    steamVD_to_turbine = PKM['steamVD_to_turbine']
    ##################################################

    # Состав газов при частичной нагрузке
    fractiongas = list(gas_streams.loc["GTU-PEVD", "N2":"Ar"])

    gas1 = prop.Materials_prop(
        gasmix,
        fractiongas,
        prop.REFPROP_h_s,
        prop.REFPROP_p_t,
        prop.REFPROP_p_h,
        prop.REFPROP_p_s,
        prop.REFPROP_p_q,
        prop.REFPROP_t_q,
        prop.REFPROP_p_rho,
        prop.REFPROP_s_q,
        RP=RP,
    )

    # Инициализаця KU+TU, она здесь потому что нжно менять состав газа на входе в КУ
    
    KU_and_TU = KU_TU.ku_tu(
        gas0,
        gas1,
        water,
        gas_streams0,
        gas_streams,
        water_streams0,
        water_streams,
        heaters,
        electric,
        "PEVD-DROSVD",
        "PPND-DROSND",
        "DROSVD-TURBVD",
        "DROSND-TURBND",
        Calcmethod,
        KPD_SP,
        KPD_KN,
        KPD_to,
        KPD_PN,
        steamVD_fraction_to_turbine,
        steamVD_to_turbine,
    )

    start_time = time.time()

    # Расчет КУ и ТУ
    KU_and_TU.calculate(
        Teplo,
        Calctolerance,
        Iterations_KU_TU,
        Iterations_cotel,
        Iterations_turbine,
    )

    print(f"fin КУ и ТУ:--- {round((time.time() - start_time), 1)} сек. ---")


calculate_CCGT(Maxiterations_KU_TU, Maxiterations_cotel, Maxiterations_turbine)

Относительная мощность ГТУ в ГТУ 1
Достигнуто максимальное количество итераций контура высокого давления
Достигнуто максимальное количество итераций контура низкого давления
Достигнуто максимальное количество итераций контура низкого давления
Повышена точность расчета котла для увеличения сходимости
dQ/Q ПЕВД+ИВД+ЭВД 2.7470752354068932
dQ/Q ПЕВД+ИВД+ЭВД -1.5694444114571808
dQ/Q ПЕВД+ИВД+ЭВД 0.7709512032806138
dQ/Q ПЕВД+ИВД+ЭВД -0.38267585265702425
dQ/Q ПЕВД+ИВД+ЭВД 0.18312903199987357
Достигнуто максимальное количество итераций контура высокого давления
dQ/Q ППНД+ИНД+ГПК 0.2637610707543666
dQ/Q ППНД+ИНД+ГПК -0.04466140194743576
Повышена точность расчета котла для увеличения сходимости
dQ/Q ПЕВД+ИВД+ЭВД -0.08191702236506469
dQ/Q ППНД+ИНД+ГПК -0.07300936356136485
Fin котел-утилизатора:--- 17.91 сек. ---
dQ/Qsumm -0.07760901428561653
Fin турбоустановка:--- 0.06 сек. ---
Fin котел-утилизатора:--- 4.57 сек. ---
dQ/Qsumm 0.009238520109017738
Fin турбоустановка:--- 0.03 сек. ---
Fin котел-ути

In [2]:
########################ОГРАНИЧЕНИЯ НА РАБОТУ ПГУ########################
# ЕСЛИ ТЕПЛОФИКАЦИЯ (из документов по ПГУ-220Т)
# Gк_мин=4,44 кг/с вроде
# Nтурбины мин = 17,6 МВт (примерно 25%)
# Максимальноа давление в отборах СП 0,245, 0,198 МПа (Теплофикационная паровая турбина Т-63/76-8.8 для серии ПГУ-230)
# ДЛЯ ВСЕХ РЕЖИМОВ (Из Трухния по ПГУ-450Т)
# Gвд_мин=25% от номинала
# Gнд_мин = 50% от номинала


GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)

Max_iterations_minimum = 10

if Сalculate_minimum == True:
    n_GTU = GTU_input.at["n", 1]
    start_time = time.time()
    for i in range(Max_iterations_minimum):
        print(f"Началась {i+1} итерация расчета ПГУ")
        if i < round(Max_iterations_minimum/2,1):
            New_iterations_KU_TU, New_iterations_cotel, New_iterations_turbine = (
                2,
                2,
                15,
            )
        else:
            New_iterations_KU_TU, New_iterations_cotel, New_iterations_turbine = (
                Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
            )

        Delt_Gcnd = 100
        Delt_Nturb = 100
        if Teplo == 1:
            Delt_Gcnd = (water_streams.at["INKOND", "G"] - 4.44) / 4.44
            Delt_Nturb = (electric.at["Turbine", "Ni"] - 17.6) / 17.6
        Delt_Gvd = (
            water_streams.at["PEVD-DROSVD", "G"] / water_streams0.at["PEVD-DROSVD", "G"]
            - 0.25
        ) / 0.25
        Delt_Gnd = (
            water_streams.at["PPND-DROSND", "G"] / water_streams0.at["PPND-DROSND", "G"]
            - 0.5
        ) / 0.5
        Delta_min = min(Delt_Gcnd, Delt_Nturb, Delt_Gvd, Delt_Gnd)
        if n_GTU == 1 and Delta_min < 0:
            print("Мощность ГТУ 100% и расход пара все еще слишком мал")
        n_GTU = n_GTU - Delta_min / 10
        GTU_input.at["n", 1] = n_GTU

        calculate_CCGT(
            New_iterations_KU_TU, New_iterations_cotel, New_iterations_turbine
        )
        print(f"Отклонение от ограничения минимальное равно {Delta_min}")
        if abs(Delta_min) < Calctolerance:
            calculate_CCGT(
            New_iterations_KU_TU, New_iterations_cotel, New_iterations_turbine
        )
            print(f"Отклонение от ограничения минимальное равно {Delta_min}")
            print(f"Относительная мощность ГТУ равна {n_GTU}")
            print(
                f"fin минимальная мощность ПГУ:--- {round((time.time() - start_time), 1)} сек. ---"
            )
            break
        if i == Max_iterations_minimum - 1:
            print("Достигнуто максимальное количество итераций минимального расхода в ПГУ", i+1)


Началась 1 итерация расчета ПГУ
Относительная мощность ГТУ в ГТУ 0.9033748202929015
Fin котел-утилизатора:--- 5.64 сек. ---
dQ/Qsumm -0.09083773564596803
Fin турбоустановка:--- 0.05 сек. ---
Достигнуто максимальное количество итераций контура высокого давления
Fin котел-утилизатора:--- 10.34 сек. ---
dQ/Qsumm -9.953130552808796e-05
Fin турбоустановка:--- 0.03 сек. ---
Достигнуто максимальное количество итераций расхода КУ+ПТУ
Error_water_G: 0.0, Error_nd_G: 0.70069534630326, Error_vd_G: 0.0
Fin котел-утилизатора:--- 1.76 сек. ---
dQ/Qsumm 0.004394244651669419
Fin турбоустановка:--- 0.03 сек. ---
Fin котел-утилизатора:--- 0.94 сек. ---
dQ/Qsumm -0.005545653269969876
Fin турбоустановка:--- 0.03 сек. ---
Максимальная погрешность определения расхода в КУ+ПТУ 0.004689870843481645
Pnd_it [0.66276]
Pvd_it [8.36303]
Время 1 итерации расчета КУ+ТУ:--- %s сек. --- 18.8
Fin котел-утилизатора:--- 7.68 сек. ---
dQ/Qsumm 0.0013326527470322786
Достигнуто максимальное количество итераций расхода и дав

Достигнуто максимальное количество итераций давления верхнего отбора
Недостаточное количество итераций
Fin турбоустановка:--- 4.51 сек. ---
Достигнуто максимальное количество итераций контура низкого давления
Fin котел-утилизатора:--- 4.02 сек. ---
dQ/Qsumm -0.006625012547194959
Fin турбоустановка:--- 1.7 сек. ---
Достигнуто максимальное количество итераций расхода КУ+ПТУ
Error_water_G: 0.0002904375558509083, Error_nd_G: 2.9922215783847865, Error_vd_G: 0.0
Fin котел-утилизатора:--- 1.75 сек. ---
dQ/Qsumm -0.000476970017031225
Fin турбоустановка:--- 0.12 сек. ---
Максимальная погрешность определения расхода в КУ+ПТУ 0.002690264440366527
Pnd_it [0.63367, 0.60834]
Pvd_it [8.25246, 8.24215]
Время 2 итерации расчета КУ+ТУ:--- %s сек. --- 33.0
Достигнуто максимальное количество итераций давления КУ+ПТУ 2
fin КУ и ТУ:--- 33.0 сек. ---
Отклонение от ограничения минимальное равно 0.5989566895557767
Началась 5 итерация расчета ПГУ
Относительная мощность ГТУ в ГТУ 0.6336517113952804
Fin котел-ути

Fin котел-утилизатора:--- 1.14 сек. ---
dQ/Qsumm 0.001889499497783761
Fin турбоустановка:--- 0.34 сек. ---
Fin котел-утилизатора:--- 0.94 сек. ---
dQ/Qsumm -0.008755849126846748
Fin турбоустановка:--- 0.11 сек. ---
Максимальная погрешность определения расхода в КУ+ПТУ 0.004016739438186528
Pnd_it [0.62641, 0.60458, 0.59579, 0.59229, 0.59088, 0.59032, 0.59012, 0.59004]
Pvd_it [8.20729, 8.19998, 8.19666, 8.19514, 8.19443, 8.19408, 8.19394, 8.1939]
Время 8 итерации расчета КУ+ТУ:--- %s сек. --- 81.6
Fin котел-утилизатора:--- 0.95 сек. ---
dQ/Qsumm 0.008504653107882638
Fin турбоустановка:--- 0.11 сек. ---
Fin котел-утилизатора:--- 0.94 сек. ---
dQ/Qsumm -0.009787772288552272
Fin турбоустановка:--- 0.11 сек. ---
Fin котел-утилизатора:--- 2.77 сек. ---
dQ/Qsumm 0.00995072732131509
Fin турбоустановка:--- 0.16 сек. ---
Fin котел-утилизатора:--- 1.85 сек. ---
dQ/Qsumm 0.007885447447044534
Fin турбоустановка:--- 0.19 сек. ---
Достигнуто максимальное количество итераций расхода КУ+ПТУ
Error_water_

Fin котел-утилизатора:--- 6.7 сек. ---
dQ/Qsumm 0.013094917022849662
Fin турбоустановка:--- 0.05 сек. ---
Достигнуто максимальное количество итераций контура высокого давления
Достигнуто максимальное количество итераций контура низкого давления
Достигнуто максимальное количество итераций контура высокого давления
Достигнуто максимальное количество итераций контура низкого давления
Достигнуто максимальное количество итераций контура высокого давления
Достигнуто максимальное количество итераций контура низкого давления
Повышена точность расчета котла для увеличения сходимости
dQ/Q ПЕВД+ИВД+ЭВД -0.9009908107161226
dQ/Q ПЕВД+ИВД+ЭВД 0.6702615831047085
dQ/Q ПЕВД+ИВД+ЭВД -0.4851820280474743
dQ/Q ПЕВД+ИВД+ЭВД 0.34164479011122956
dQ/Q ПЕВД+ИВД+ЭВД -0.25414951429152255
Достигнуто максимальное количество итераций контура высокого давления
dQ/Q ППНД+ИНД+ГПК -0.36388922302715937
dQ/Q ППНД+ИНД+ГПК 0.10813112642994055
dQ/Q ППНД+ИНД+ГПК -0.03203094117007536
dQ/Q ППНД+ИНД+ГПК 0.00827225971440304
Повыш

Fin котел-утилизатора:--- 0.95 сек. ---
dQ/Qsumm -0.004783143628787397
Fin турбоустановка:--- 0.11 сек. ---
Fin котел-утилизатора:--- 0.99 сек. ---
dQ/Qsumm 0.0012808635051470801
Fin турбоустановка:--- 0.12 сек. ---
Максимальная погрешность определения расхода в КУ+ПТУ 0.005250216434181446
Pnd_it [0.62384, 0.59802, 0.58731, 0.58276, 0.58096, 0.58018, 0.57986, 0.57973, 0.57967, 0.57957, 0.57954]
Pvd_it [8.05678, 8.00085, 7.975, 7.96307, 7.95751, 7.95497, 7.95377, 7.95321, 7.95296, 7.95291, 7.95283]
Время 11 итерации расчета КУ+ТУ:--- %s сек. --- 146.5
Достигнуто максимальное количество итераций контура высокого давления
Fin котел-утилизатора:--- 4.76 сек. ---
dQ/Qsumm 0.005293207235527312
Fin турбоустановка:--- 0.39 сек. ---
Fin котел-утилизатора:--- 2.29 сек. ---
dQ/Qsumm -0.0006224007034123019
Fin турбоустановка:--- 0.61 сек. ---
Fin котел-утилизатора:--- 1.84 сек. ---
dQ/Qsumm 0.002598169168575759
Fin турбоустановка:--- 0.12 сек. ---
Максимальная погрешность определения расхода в КУ+

KeyboardInterrupt: 

In [ ]:
# Вывод газовых потоков
gas_streams

In [ ]:
# Вывод водяных потоков
water_streams

In [ ]:
Errors_water = abs(water_streams - water_streams0) / water_streams0 * 100
Errors_gas = abs(gas_streams - gas_streams0) / gas_streams0 * 100
Errors_gas
Errors_water

In [ ]:
gas_streams0

In [ ]:
electric

In [ ]:
heaters

In [ ]:
GTU_input

In [ ]:
import Grafiki

Grafiki.Q_t_diagram(gas_streams, water_streams, heaters, 15, 10)
Grafiki.H_S_diagram(water, water_streams, 15, 10)